In [3]:
import pandas as pd

df = pd.read_csv('strong.csv')

In [8]:
import re
splitPattern = re.compile(r'[^a-zA-Z0-9]+') 
def rename(col:str)->str:
    words = splitPattern.split(col)
    return words[0].lower() + ''.join(w.capitalize() for w in words[1:])

if not globals().get('__renamed'):
    __renamed = True
    df.columns = df.columns.map(rename)

df.exerciseName.value_counts()

{
    'Squat': ['quad', 'hamstring', 'glute'],
    'Deadlift': ['hamstring', 'glute', 'back'],
    'Bench': ['chest', 'tricep', 'shoulder'],
    'Overhead Press': ['shoulder', 'tricep', 'Traps'],
    'Row': ['rhomboids', 'bicep', 'lats'],
}


exerciseName
Squat (Barbell)                     439
Deadlift (Barbell)                  331
Overhead Press (Barbell)            323
Chest Dip                           246
Pull Up                             146
                                   ... 
Walking Lunge                         2
Glute Ham Raise                       2
Hip Extension                         2
Hack Squat                            2
Seated Calf Raise (Plate Loaded)      2
Name: count, Length: 113, dtype: int64

In [74]:
with open('exercise_muscles_worked.txt','r') as f:
    html = f.read()

rowPattern = re.compile(r'<tr class="border-gray-200 border">.*?</tr>', re.DOTALL)
exerciseNamePattern = re.compile((
                                    r'<tr class="border-gray-200 border"><td class="whitespace-normal  py-3 pl-4 pr-3 '
                                    r'font-medium text-gray-900 sm:pl-3 border-t w-1/5  text-xs sm:text-base"><a href="'
                                    r'[a-zA-Z0-9\s\.\(\)\-\\/]+">(?P<exercise>[a-zA-Z0-9\s\.\(\)\-\']+)</a></td><td class='
                                    r'"whitespace-normal.*?')
                                )
headingsPattern = re.compile(r'<tr class="border-t border-gray-200">.*?</tr>', re.DOTALL)
musclesPattern = re.compile((
                                r'<tr class="border-t border-gray-200"><th colspan="4" scope="colgroup" class="bg-mw-blu'
                                r'e py-2 pl-4 pr-3 text-left text-base font-semibold text-white sm:pl-3">'
                                r'(?P<muscle>[a-zA-Z0-9\s\.\(\)\-\']+)</th></tr>'
                            ))

data = {}
for group,header in zip(headingsPattern.split(html)[1:],headingsPattern.findall(html)):
    muscle = musclesPattern.search(header).group('muscle')
    exercises = exerciseNamePattern.findall(group)
    data[muscle] = exercises

In [81]:
# exerciseNamePattern.findall(group)
muscleExercise = pd.DataFrame([dict(muscle = muscle,exercise = exercise) for muscle,exercises in data.items() for exercise in exercises])

In [83]:
muscleExercise

,muscle,exercise
0,Biceps,Barbell Bent Over Row
1,Biceps,Barbell Reverse Curl
2,Biceps,Barbell Landmine Row
3,Biceps,Barbell Long Landmine Row
4,Biceps,Barbell Meadows Row
...,...,...
3181,Groin,Bodyweight Butterfly Stretch
3182,Groin,Machine Hip Adduction
3183,Groin,Machine Hip And Glute Adduction
3184,Groin,Cable Standing Hip Adduction


In [98]:
strongExerciseNames

,exercise_strong
0,Lat Pulldown (Single Arm)
1,Pull Up
2,Bent Over One Arm Row (Dumbbell)
3,Iso-Lateral Row (Machine)
4,Reverse Fly (Machine)
...,...
108,Tricep Press
109,Walking Lunge BB
110,Straight Arm Lat Pulldown
111,Incline Bench Press (Smith Machine)


In [102]:
def removeParentheses(s:str)->str:
    return re.sub(r'\(.*?\)','',s)
def makeTitle(s:str)->str:
    return s.title()
def removeSpecial(s:str)->str:
    return re.sub(r'[^a-zA-Z0-9]','',s)
def clean(s:str)->str:
    return removeSpecial(makeTitle(removeParentheses(s)))
strongExerciseNames = pd.DataFrame(data = dict(exercise_strong = df.exerciseName.unique()))
muscleExercise['exercise_clean'] = muscleExercise.exercise.apply(clean)
strongExerciseNames['exercise_clean'] = strongExerciseNames.exercise_strong.apply(clean)

In [117]:
(
strongExerciseNames.merge(muscleExercise, on = 'exercise_clean', how = 'outer', indicator = True)
    .query('_merge == "left_only"')#.exercise_clean.unique()
    # .query("_merge == 'right_only'")#.exercise_clean.unique()
    # .query("_merge == 'both'")#.exercise_clean.unique()
)

,exercise_strong,exercise_clean,muscle,exercise,_merge
0,Ab Wheel,AbWheel,NaN,NaN,left_only
35,Back Extension,BackExtension,NaN,NaN,left_only
396,Bench Press (Smith Machine),BenchPress,NaN,NaN,left_only
397,Bench Press (Barbell),BenchPress,NaN,NaN,left_only
398,Bench Press (Dumbbell),BenchPress,NaN,NaN,left_only
...,...,...,...,...,...
3116,Triceps Pushdown (Cable - Straight Bar),TricepsPushdown,NaN,NaN,left_only
3191,Upright Row (Cable),UprightRow,NaN,NaN,left_only
3192,Upright Row (Barbell),UprightRow,NaN,NaN,left_only
3193,Upright Row (Dumbbell),UprightRow,NaN,NaN,left_only


In [ ]:
(
    muscleExercise.exercise.apply(lambda s: pd.Series(dict(lift = s,super = s.split()[-1])))
    .assign(variant = lambda df: df.lift.apply(lambda s: s.split()[0]))
)

,lift,super,variant
0,Barbell Bent Over Row,Row,Barbell
1,Barbell Reverse Curl,Curl,Barbell
2,Barbell Landmine Row,Row,Barbell
3,Barbell Long Landmine Row,Row,Barbell
4,Barbell Meadows Row,Row,Barbell
...,...,...,...
3181,Bodyweight Butterfly Stretch,Stretch,Bodyweight
3182,Machine Hip Adduction,Adduction,Machine
3183,Machine Hip And Glute Adduction,Adduction,Machine
3184,Cable Standing Hip Adduction,Adduction,Cable


: 